In [1]:
import pandas as pd
import numpy as np
from gmf import GMF
from mlp import MLP
from neumf import NeuMF
from neumf_social2 import NeuMF_Social_2
import torch.optim as optim
import math
from model_utils import *

In [2]:
data_path = "./data/yelp2/"

In [3]:
train_ratings = list(np.load(data_path + "train_ratings.npy"))
val_ratings = list(np.load(data_path + "val_ratings.npy"))
test_ratings = list(np.load(data_path + "test_ratings.npy"))

In [4]:
social_embeddings_dict = np.load(data_path + "node2vec_embeddings.npy").item()

In [5]:
yelp_rating_path = data_path + 'yelp.train.rating'
yelp_ratings = pd.read_csv(yelp_rating_path, sep='\t', header=None, names=['uid', 'rid', 'rating'],  engine='python')
user_id = yelp_ratings[['uid']].drop_duplicates()
item_id = yelp_ratings[['rid']].drop_duplicates()
num_users = user_id['uid'].max() - user_id['uid'].min() + 1
num_items = item_id['rid'].max() - item_id['rid'].min() + 1

In [6]:
neumf_social_config = {'alias': 'pretrain_neumf_factor8neg4',
                'num_epoch': 25,
                'batch_size': 1024,
                'num_users': num_users,
                'num_items': num_items,
                'latent_dim_mf': 16,
                'latent_dim_mlp': 16,
                'layers': [32,16,16],  # 32 layers[0] is the concat of latent user vector & latent item vector
                'social_layers': [128,32,16],
                'pretrain': False,
                'pretrain_mf': None,
                'pretrain_mlp': None
                }

In [7]:
training_params = {'batch_size': 1024,'shuffle': True,'num_workers': 0}
training_set = Dataset(train_ratings, social_embeddings_dict)
train_generator = data.DataLoader(training_set, **training_params)
val_params = {'batch_size': 1024,'shuffle': True,'num_workers': 0}
val_set = Dataset(val_ratings, social_embeddings_dict)
val_generator = data.DataLoader(val_set, **val_params)
test_params = {'batch_size': 1024,'shuffle': True,'num_workers': 0}
test_set = Dataset(test_ratings, social_embeddings_dict)
test_generator = data.DataLoader(test_set, **test_params)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
def train_neumf_social():
    model = NeuMF_Social_2(neumf_social_config).to(device)
#     if config['pretrain']:  #TODO:: Manoj
#         model.load_pretrain_weights()
    opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=1e-3)
    criterion = torch.nn.MSELoss()
    for epoch in range(neumf_social_config['num_epoch']):
        print("running epoch ", epoch)
        train_mse = epoch_run(model, train_generator, opt,criterion, "train")
        val_mse = epoch_run(model, val_generator, opt,criterion, "val")
        print("train mse loss => ", train_mse, "val mse loss => ", val_mse)
    return model

In [10]:
def epoch_run(model, generator, opt, criterion,mode="train", social=False):
    running_loss = 0
    if(mode == "train"):
        model.train()
    else:
        model.eval()
    for local_batch, local_labels in generator:
        local_batch_social  = torch.tensor(local_batch[:, 2:]).type(torch.float).to(device)
        local_batch  = torch.tensor(local_batch[:, 0:2]).type(torch.long).to(device)
        local_labels = local_labels.type(torch.float).to(device)
        #print(local_batch.size(), local_batch_social.size())print()
        #print(local_batch_social[0:2])
        y_preds = model(local_batch[:,0], local_batch[:,1], local_batch_social)
        loss = criterion(y_preds, local_labels)
        running_loss += (loss.item()*local_labels.size()[0])
        if(mode == "train"):
            opt.zero_grad()
            loss.backward()
            opt.step()
    avg_loss = running_loss * 1.0 / (len(generator.dataset))
    return avg_loss

In [11]:
neumf_model = train_neumf_social()

running epoch  0


/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


train mse loss =>  2.2260007730437144 val mse loss =>  1.6120024621575488
running epoch  1
train mse loss =>  1.5816949237345557 val mse loss =>  1.5721298078850765
running epoch  2
train mse loss =>  1.490832807102845 val mse loss =>  1.4475319530670383
running epoch  3
train mse loss =>  1.4056148395776251 val mse loss =>  1.3741927932647218
running epoch  4
train mse loss =>  1.3264220271771916 val mse loss =>  1.3273381083071858
running epoch  5
train mse loss =>  1.2748667018577788 val mse loss =>  1.3086775742650851
running epoch  6
train mse loss =>  1.2529628300058953 val mse loss =>  1.3028345393275362
running epoch  7
train mse loss =>  1.2426390310260844 val mse loss =>  1.2987778189820502
running epoch  8
train mse loss =>  1.2380602597793773 val mse loss =>  1.3001947607838185
running epoch  9
train mse loss =>  1.2335695235093864 val mse loss =>  1.3003243631608437
running epoch  10
train mse loss =>  1.2297311137929385 val mse loss =>  1.299778223464078
running epoch  11

In [13]:
torch.save(neumf_model.state_dict(), "./saved_models/neumf_social_2.dict")

In [14]:
model = NeuMF_Social_2(neumf_social_config).to(device)
model.load_state_dict(torch.load("./saved_models/neumf_social_2.dict"))
model.eval()

NeuMF_Social_2(
  (embedding_user_mlp): Embedding(122824, 16)
  (embedding_item_mlp): Embedding(28007, 16)
  (embedding_user_mf): Embedding(122824, 16)
  (embedding_item_mf): Embedding(28007, 16)
  (embedding_item_social): Embedding(28007, 64)
  (fc_social_layers): ModuleList(
    (0): Linear(in_features=128, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=16, bias=True)
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=16, bias=True)
  )
  (affine_output): Linear(in_features=48, out_features=1, bias=True)
)

In [15]:
def predict(model, generator):
    model.eval()
    y_preds_all = torch.Tensor().to(device) 
    y_labels_all = torch.Tensor().to(device) 
    for local_batch, local_labels in generator:
        local_batch_social  = torch.tensor(local_batch[:, 2:]).type(torch.float).to(device)
        local_batch  = torch.tensor(local_batch[:, 0:2]).type(torch.long).to(device)
        local_labels = local_labels.type(torch.float).to(device)
        with torch.no_grad():
            y_preds = model(local_batch[:,0], local_batch[:,1], local_batch_social)
        y_preds_all = torch.cat((y_preds_all,y_preds))
        y_labels_all = torch.cat((y_labels_all,local_labels))
    return y_preds_all, y_labels_all

def evaluate(model, generator):
    y_preds_all, y_labels_all = predict(model, generator)  
    y_preds = list(y_preds_all.view(1, y_preds_all.size()[0]).to("cpu").numpy()[0])
    y_actuals = list(y_labels_all.view(1, y_labels_all.size()[0]).to("cpu").numpy()[0])
    #print(type(y_preds), type(y_actuals))
    tmse = sum([(a-b) * (a-b) for a,b in zip(y_preds, y_actuals)])
    rmse = math.sqrt((1.0*tmse)/len(y_preds))
    return rmse

In [16]:
evaluate(model, test_generator)

/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


1.1375855525169463